# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [2]:
dataset = load_adult()
dataset.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [3]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,0.301370,0.833333,0.000000,0.333333,0.615385,0.6,0.0,1.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.000000,0.000000,0.307692,0.4,0.0,1.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.200000,0.166667,0.461538,0.6,0.0,1.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.133333,0.000000,0.461538,0.4,1.0,1.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.000000,0.000000,0.384615,0.0,1.0,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [4]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 28162
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [5]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[6, 4], [6, 10], [6, 8], [6, 2], [6, 3], [0, 4], [0, 8], [0, 10], [0, 1], [0, 3], [0, 2], [0, 5], [7, 4], [7, 3], [7, 10], [7, 1], [7, 2], [7, 5], [9, 3], [9, 8], [9, 2], [9, 1], [9, 10], [9, 5], [3, 4], [3, 8], [3, 10], [3, 1], [3, 5], [3, 2], [2, 4], [2, 8], [2, 10], [2, 1], [2, 5], [4, 10], [8, 10], [1, 10], [5, 10]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [6]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {10: [7]}
Bias dict DP: {10: [4, 8, 3, 2, 7, 1, 5]}
Bias dict CF: {10: [3, 7]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [7]:
X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
base_clf = MLPClassifier()
base_clf.fit(X_train, y_train)

def eval_model(dataset_train, dataset_test, use_base_clf=True):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    if use_base_clf:
        y_train = base_clf.predict(X_train)

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [8]:
eval_model(dataset_train, dataset_test, use_base_clf=False)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8641509433962264,
 'recall': 0.914780292942743,
 'auroc': 0.7405226765918533,
 'dp': 0.19098378858538756,
 'ftu': 0.10099999999999998}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [9]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2022-01-26 15:45:57.822107: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-26 15:45:57.901774: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
 10%|█         | 1/10 [00:08<01:19,  8.82s/it]

0 [D loss: 0.000306, acc.: 100.00%] [G loss: 25.606836]
generated_data


 20%|██        | 2/10 [00:18<01:12,  9.10s/it]

1 [D loss: 0.000445, acc.: 100.00%] [G loss: 50.612782]


 30%|███       | 3/10 [00:25<00:59,  8.54s/it]

...

8 [D loss: 0.000018, acc.: 100.00%] [G loss: 162.503632]


100%|██████████| 10/10 [01:24<00:00,  8.45s/it]


9 [D loss: 0.000023, acc.: 100.00%] [G loss: 149.851135]


Synthetic data generation: 100%|██████████| 221/221 [00:01<00:00, 114.77it/s]


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,0.016644,0.500000,0.133333,0.333333,0.230769,0.6,1.00,0.0,0.041676,0.625,1.0
1,0.017935,0.166667,0.733333,1.000000,0.000000,0.0,0.25,1.0,0.036969,0.800,0.0
2,0.024360,1.000000,0.333333,0.666667,0.076923,0.0,0.25,1.0,0.051346,0.725,1.0
3,0.022331,0.333333,0.933333,0.500000,0.769231,1.0,0.00,0.0,0.039752,0.300,1.0
4,0.015566,0.000000,0.733333,0.166667,0.846154,0.6,0.00,1.0,0.036299,0.650,0.0


In [10]:
eval_model(synth_data, dataset_test)

{'precision': 0.751, 'recall': 1.0, 'auroc': 0.5, 'dp': 0.0, 'ftu': 0.0}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

 10%|█         | 1/10 [00:04<00:43,  4.87s/it]

Epoch: 0 | disc_loss: 0.3266867697238922 | gen_loss: -0.05095595866441727


 20%|██        | 2/10 [00:07<00:30,  3.82s/it]

Epoch: 1 | disc_loss: 0.4819841980934143 | gen_loss: -0.05299055948853493


 30%|███       | 3/10 [00:10<00:23,  3.41s/it]

...

Epoch: 8 | disc_loss: 4.909919738769531 | gen_loss: 0.04439130052924156


100%|██████████| 10/10 [00:28<00:00,  2.89s/it]


Epoch: 9 | disc_loss: 0.06361019611358643 | gen_loss: 0.05286000669002533


Synthetic data generation: 100%|██████████| 57/57 [00:00<00:00, 72.69it/s]


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,0.216853,0.833333,0.866667,1.000000,0.461538,0.0,0.25,1.0,0.269746,0.975,0.0
1,0.161406,0.166667,0.933333,1.000000,0.384615,1.0,0.75,0.0,0.218034,0.200,0.0
2,0.164138,0.500000,0.466667,0.500000,0.384615,0.0,0.25,1.0,0.207796,0.025,0.0
3,0.211258,0.166667,1.000000,0.500000,0.923077,1.0,0.50,0.0,0.225394,0.550,0.0
4,0.180762,1.000000,0.000000,0.333333,0.076923,0.2,0.50,0.0,0.253279,1.000,0.0


In [12]:
eval_model(synth_data, dataset_test)

{'precision': 0.8430493273542601,
 'recall': 0.8761651131824234,
 'auroc': 0.6920986208482398,
 'dp': 0.16129062106743652,
 'ftu': 0.009000000000000008}

### FairGAN

In [13]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
cache/adult.npy


2022-01-26 15:48:06.187116: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


Pretrain_Epoch:0, trainLoss:0.076982, validLoss:0.067736, validReverseLoss:0.000000
Pretrain_Epoch:1, trainLoss:0.067206, validLoss:0.067667, validReverseLoss:0.000000
Pretrain_Epoch:2, trainLoss:0.067199, validLoss:0.067599, validReverseLoss:0.000000
Pretrain_Epoch:3, trainLoss:0.067190, validLoss:0.067722, validReverseLoss:0.000000
Pretrain_Epoch:4, trainLoss:0.067178, validLoss:0.067702, validReverseLoss:0.000000
Pretrain_Epoch:5, trainLoss:0.067164, validLoss:0.067628, validReverseLoss:0.000000
Pretrain_Epoch:6, trainLoss:0.067195, validLoss:0.067693, validReverseLoss:0.000000
Pretrain_Epoch:7, trainLoss:0.067164, validLoss:0.067477, validReverseLoss:0.000000
Pretrain_Epoch:8, trainLoss:0.065271, validLoss:0.061420, validReverseLoss:0.000000
Pretrain_Epoch:9, trainLoss:0.057910, validLoss:0.058134, validReverseLoss:0.000000
Epoch:0, d_loss:0.446370, g_loss:2.482316, d accuracy:0.932241, d AUC:0.999179, g accuracy:0.865862, rdf 0.000000
Epoch:1, d_loss:0.020031, g_loss:5.544989, d a

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,0.000000,0.648793,0.000000,0.418355,0.547123,0.00000,0.962435,0.453676,0.000000,1.187743,1.0
1,0.271561,0.271027,0.000000,0.244098,0.572727,0.00000,0.437914,0.261232,0.309797,0.561148,1.0
2,0.404406,0.180820,0.000000,0.430635,0.607781,0.00000,0.516901,0.498149,0.000000,0.591935,1.0
3,0.000000,0.123709,0.040932,0.284229,0.708794,0.33513,0.887392,0.479269,0.140241,0.808210,1.0
4,0.000000,0.503142,0.000000,0.437908,0.485040,0.00000,0.307282,0.311601,0.000000,0.708190,1.0


In [14]:
eval_model(synth_data, dataset_test, use_base_clf=False)

{'precision': 0.751,
 'recall': 1.0,
 'auroc': 0.5,
 'dp': 0.0,
 'ftu': 0.0004999999999999449}

### DECAF

#### DECAF-ND

In [15]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipp

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Epoch 9: 100%|██████████| 441/441 [00:08<00:00, 50.58it/s, loss=-0.187] 


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
31098,0.263480,1.507316e-03,0.110036,0.008862,0.715060,0.472428,2.914393e-20,1.000000,0.504632,2.246149e-08,1.0
13220,0.069070,1.216912e-04,0.553274,0.528238,0.111467,0.445988,2.284749e-22,0.037209,0.067462,2.254699e-05,0.0
12494,0.035345,4.628779e-05,0.549271,0.081706,0.049620,0.475040,3.547936e-13,1.000000,0.032831,2.077963e-03,0.0
5818,0.015769,4.723412e-09,0.023783,0.054339,0.113030,0.475854,1.529863e-03,1.000000,0.265762,1.170837e-02,1.0
26344,0.066809,6.144223e-10,0.525036,0.568093,0.035190,0.772796,9.847660e-01,0.003022,0.349117,2.821043e-03,1.0


In [16]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8332344213649852,
 'recall': 0.9347536617842876,
 'auroc': 0.6852483168359189,
 'dp': 0.19556406839884521,
 'ftu': 0.09750000000000003}

#### DECAF-FTU

In [17]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
31098,0.513834,3.465279e-05,0.043836,0.003029,0.053156,0.476351,1.167375e-16,1.000000,0.504541,8.992860e-10,1.0
13220,0.513656,3.736998e-08,0.070198,0.001238,0.733068,0.475479,1.040192e-08,1.000000,0.504525,9.164992e-04,0.0
12494,0.011761,2.234878e-03,0.001224,0.015197,0.210336,0.474551,1.215260e-02,1.000000,0.050269,3.431812e-04,0.0
5818,0.513505,5.431365e-01,0.005103,0.545531,0.126419,0.631046,1.750929e-06,0.003078,0.025398,5.073954e-08,1.0
26344,0.165430,1.326380e-01,0.181701,0.008343,0.029924,0.476140,3.439519e-01,0.999980,0.429768,3.911569e-09,0.0


In [18]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8321554770318021,
 'recall': 0.940745672436751,
 'auroc': 0.6842282579051224,
 'dp': 0.1837478717891775,
 'ftu': 0.09150000000000003}

#### DECAF-CF

In [19]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
31098,0.513879,1.508410e-09,0.478396,0.009639,0.132774,0.523466,4.918117e-08,1.000000,0.187101,1.279412e-08,0.0
13220,0.455650,2.307447e-07,0.003080,0.257310,0.372721,0.449743,5.480567e-18,0.003312,0.504076,8.028973e-05,1.0
12494,0.513551,5.569239e-06,0.021218,0.069864,0.759047,0.473110,9.356151e-01,1.000000,0.274506,1.923744e-05,0.0
5818,0.081930,1.449035e-10,0.001989,0.327596,0.753557,0.751467,9.190407e-01,1.000000,0.058415,1.525295e-04,1.0
26344,0.090756,8.855354e-08,0.513477,0.019586,0.032919,0.455501,6.922327e-07,0.003043,0.304622,2.425330e-04,1.0


In [20]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8398791540785498,
 'recall': 0.9254327563249002,
 'auroc': 0.6966521211343377,
 'dp': 0.2151806203271892,
 'ftu': 0.10399999999999998}

#### DECAF-DP

In [21]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
31098,0.513597,2.539681e-06,0.009175,0.010815,0.697393,0.452946,5.100411e-03,0.003426,0.504155,0.079307,0.0
13220,0.514042,1.124403e-08,0.548260,0.124423,0.034891,0.645745,7.695490e-22,0.998870,0.272800,0.003033,0.0
12494,0.513680,5.425395e-06,0.542363,0.051992,0.036234,0.475916,9.835557e-01,1.000000,0.503772,0.000047,1.0
5818,0.514040,2.604779e-05,0.550939,0.006742,0.236176,0.567472,1.131644e-17,1.000000,0.423190,0.000006,0.0
26344,0.513988,2.848393e-11,0.555618,0.551892,0.139312,0.444078,1.478075e-07,0.002957,0.398821,0.004892,1.0


In [22]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8409915356711004,
 'recall': 0.9260985352862849,
 'auroc': 0.698993042743544,
 'dp': 0.21128506921311718,
 'ftu': 0.08700000000000008}